<a href="https://colab.research.google.com/github/ArjunJagdale/VoiceRecog/blob/main/VoiceRecog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gradio Prototype

In [1]:
!pip install gradio

In [2]:
!pip install librosa resemblyzer numpy soundfile

In [3]:
import gradio as gr
import os
import numpy as np
import librosa
from resemblyzer import VoiceEncoder
import soundfile as sf

In [5]:
def load_audio(file, target_sr=16000):
    """Loads an audio file and resamples it to the target sample rate."""
    audio, sr = librosa.load(file, sr=target_sr)
    return audio

In [6]:
def extract_embeddings(encoder, audio_files):
    """Extracts voice embeddings from uploaded audio files."""
    embeddings = []
    for file_path in audio_files:
        audio = load_audio(file_path)  # Load and preprocess the audio file
        embedding = encoder.embed_utterance(audio)  # Extract voice embedding
        embeddings.append(embedding)
    return np.array(embeddings)

In [7]:
def compute_similarity(embedding, reference_embeddings):
    """Computes similarity between test and reference embeddings."""
    similarities = np.dot(reference_embeddings, embedding) / (
        np.linalg.norm(reference_embeddings, axis=1) * np.linalg.norm(embedding)
    )
    return np.mean(similarities)

In [8]:
encoder = VoiceEncoder()
reference_embeddings = []

/usr/local/lib/python3.11/dist-packages/resemblyzer/voice_encoder.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_fpath, map_location="c

Loaded the voice encoder model on cuda in 0.36 seconds.


In [9]:
def train_voice_samples(files):
    global reference_embeddings
    if len(files) > 50:
        return "Please upload up to 50 files only."

    reference_embeddings = extract_embeddings(encoder, [file.name for file in files])
    return f"Extracted embeddings from {len(files)} voice samples. Ready for testing!"

In [10]:
def test_voice(file):
    try:
        if reference_embeddings is None or len(reference_embeddings) == 0:
            return "No reference voice samples found. Please upload training samples first."

        # Debugging: Check if file is received
        print(f"Received test file: {file.name}")

        # Load test audio properly
        test_audio, sr = librosa.load(file.name, sr=16000)

        # Debugging: Check audio shape
        print(f"Loaded test audio, shape: {test_audio.shape}, Sample rate: {sr}")

        # Extract embedding
        test_embedding = encoder.embed_utterance(test_audio)

        # Compute similarity
        similarity_score = compute_similarity(test_embedding, reference_embeddings)

        # Debugging: Check similarity score
        print(f"Computed similarity score: {similarity_score}")

        # Generate result message
        result = f"Similarity Score: {similarity_score:.2f}\n"
        if similarity_score > 0.8:
            result += "The voice matches closely with the training samples!"
        else:
            result += "The voice does not match the training samples."
        return result

    except Exception as e:
        return f"Error: {str(e)}"


In [11]:
with gr.Blocks() as app:
    gr.Markdown("## Voice Recognition with Similarity Testing")
    with gr.Row():
        # Change here: Use file_count instead of multiple
        train_audio = gr.File(label="Upload up to 50 training voice samples", file_types=[".wav"], file_count="multiple")
        train_button = gr.Button("Train Model")
    train_output = gr.Textbox()
    train_button.click(train_voice_samples, inputs=train_audio, outputs=train_output)

    with gr.Row():
        test_audio = gr.File(label="Upload a test voice file", file_types=[".wav"])
        test_button = gr.Button("Test Voice")
    test_output = gr.Textbox()
    test_button.click(test_voice, inputs=test_audio, outputs=test_output)

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b39c1a9d592a836600.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
